In [2]:
import os

import deepl
import pandas as pd
from dotenv import load_dotenv
from tqdm import tqdm
import tiktoken
from openai import OpenAI
import faiss
import numpy as np


# Enable tqdm for pandas
tqdm.pandas()

In [3]:
load_dotenv("./../.env")

True

In [4]:
openai_api_key = os.getenv("OPENAI_API_KEY")

In [5]:
client = OpenAI(api_key=openai_api_key)

def get_embedding(text, model="text-embedding-ada-002"):
    response = client.embeddings.create(input=text, model=model)
    return response.data[0].embedding

model = "text-embedding-ada-002"

In [6]:
index = faiss.read_index('../datasets/math-500-uk.index')
inference_df = pd.read_csv('../datasets/math-500-uk-inference.csv')
test_df = pd.read_csv('../datasets/math-500-uk-test.csv')

In [26]:
def similarity_search(query: str, top_k: int, similarity_threshold: float = 0.7) -> pd.DataFrame:
    query_embedding = np.array(get_embedding(query, model=model)).astype('float32').reshape(1, -1)

    # Normalize the query embedding
    faiss.normalize_L2(query_embedding)

    # Perform the search to retrieve top k similar embeddings
    distances, indices = index.search(query_embedding, top_k)

    # Convert distances to cosine similarities
    # Since embeddings are normalized, the inner product is equivalent to cosine similarity
    cosine_similarities = distances[0]


    # Filter results based on the similarity threshold
    filtered_indices = indices[0][cosine_similarities >= similarity_threshold]

    # Retrieve corresponding texts
    similar_texts = inference_df.iloc[filtered_indices]

    return similar_texts

similar_examples_df = similarity_search(
    query="Задача по геометрії",
    top_k=25,
    similarity_threshold=0.75
)
similar_examples_df

,problem,solution,answer,subject,level,unique_id,original_level,problem_char_count,solution_char_count,subject_char_count,...,problem_translated,solution_translated,subject_translated,level_translated,problem_translated_token_count,solution_translated_token_count,subject_translated_token_count,level_translated_token_count,embedding_text,embedding
222,The area of $\triangle ABC$ is 6 square centim...,"Since $AB \parallel DE,$ we know that $\angle ...",54,Geometry,High-Level Math Competition Problems; Regional...,test/geometry/547.json,4,342,424,8,...,Площа $\трикутника ABC$ дорівнює 6 квадратних ...,"Оскільки $AB \паралельний DE,$ ми знаємо, що $...",Геометрія,Математичні олімпіадні задачі високого рівня; ...,176,272,7,50,Математичні олімпіадні задачі високого рівня; ...,"[0.010712559334933758, -0.007907348684966564, ..."
308,Let $ABCD$ be a regular tetrahedron with side ...,The plane intersects each face of the tetrahed...,1+2\sqrt{3},Geometry,Advanced Math Competitions; International Olym...,test/geometry/711.json,5,203,375,8,...,Нехай $ABCD$ - правильний тетраедр зі стороною...,Площина перетинає кожну грань тетраедра по сер...,Геометрія,Поглиблені математичні змагання; міжнародна ол...,117,191,7,56,Поглиблені математичні змагання; міжнародна ол...,"[0.008417361415922642, 0.000720419455319643, 5..."
299,"Consider the rectangle with vertices at $(5,4)...",Points with integer coordinates are called lat...,63,Geometry,High-Level Math Competition Problems; Regional...,test/geometry/1097.json,4,156,387,8,...,"Розглянемо прямокутник з вершинами $(5,4),$ $(...",Точки з цілочисельними координатами називаютьс...,Геометрія,Математичні олімпіадні задачі високого рівня; ...,85,209,7,50,Математичні олімпіадні задачі високого рівня; ...,"[-0.0031878254376351833, -0.003637160174548626..."
169,What is the number of square units in the area...,"This triangle is isosceles, and so the altitud...",12,Geometry,Foundational High-School Math,test/geometry/802.json,2,98,337,8,...,Скільки квадратних одиниць міститься у площі т...,"Цей трикутник рівнобедрений, тому висота, пров...",Геометрія,Фундаментальна математика для середньої школи,62,225,7,23,Фундаментальна математика для середньої школи\...,"[0.021239973604679108, 0.022146791219711304, 0..."
123,"In the diagram, $D$ and $E$ are the midpoints ...",$\triangle DBC$ has base $\overline{BC}$ of le...,8,Geometry,High-Level Math Competition Problems; Regional...,test/geometry/283.json,4,568,587,8,...,На рисунку $D$ і $E$ - середини відрізків $\ov...,Трикутник $\triangle DBC$ має основу $\overlin...,Геометрія,Математичні олімпіадні задачі високого рівня; ...,271,318,7,50,Математичні олімпіадні задачі високого рівня; ...,"[0.008777080103754997, -0.00464252894744277, -..."
152,"The medians $AD$, $BE$, and $CF$ of triangle $...","Since $E$ is the midpoint of $AC$, the area of...",8,Geometry,Advanced Math Competitions; International Olym...,test/geometry/172.json,5,260,873,8,...,"Медіани $AD$, $BE$ і $CF$ трикутника $ABC$ пер...","Оскільки $E$ є серединою $AC$, то площа трикут...",Геометрія,Поглиблені математичні змагання; міжнародна ол...,145,453,7,56,Поглиблені математичні змагання; міжнародна ол...,"[0.010307596065104008, -0.0026040917728096247,..."
154,"A gecko is in a room that is 12 feet long, 10 ...",[asy]\nimport three;\ncurrentprojection=orthog...,2\sqrt{113},Geometry,Advanced Math Competitions; International Olym...,test/geometry/880.json,5,546,3356,8,...,"Гекон знаходиться у кімнаті довжиною 12 футів,...",[asy]\nімпортувати три;\ncurrentprojection=ort...,Геометрія,Поглиблені математичні змагання; міжнародна ол...,329,1887,7,56,Поглиблені математичні змагання; міжнародна ол...,"[-0.005317579489201307, 0.01098331343382597, -..."
100,A hexagon is inscribed in a circle: [asy]\npai...,"Labeling our vertices will help a great deal, ...",145^\circ,Geometry,Advanced Math Competitions; International Olym...,test/geometry/183.json,5,409,1259,8,...,"У коло вписано шестикутник: [asy].\nпара pA, p...","Позна

In [15]:
system_prompt = """
Ти корисний помічник для генерації задач з математики українською мовою.
Маючи вхідний запит користувача ("USER_INPUT"), який включає тему, чи ключові слова, та знайдені схожі приклади задач ("SIMILAR_EXAMPLES") згенеруй 5 задач на тему, якщо користувач не попросить інакше.
Візьми до уваги схожі теми задач, та тему з питання якщо така задана.
Візьми до уваги потрібну складність задачі, якщо така задана. Якщо ж ні, спробуй згенерувати простіші завдання, вибираючи простіші приклади.
Текст у прикладах ("SIMILAR_EXAMPLES") може мати помилки у LaTeX синтаксисі після перекладу, спробуй їх виправити у відповіді, щоб згенерувати валідну відповідь.
Згенеруй 5 прикладів задач з покроковими розв’язками, уникаючи помилок.
Використовуй увесь доступний контекст для більшої точності та різноманітності.
Спробуй генерувати різноманітні задачі.
Якщо ви генеруєте код LaTeX або рівняння, використовуйте формат $ замість W( і $$ замість \\I; замість багаторядкового LaTeX у форматі
[
довільне рівняння
]
генеруй $$
довільне LaTeX рівняння для правильного трактування у Markdown форматі
$$
Також візьми до уваги, що при генерації коду чи рівняння LaTeX у однорядковому форматі, замість ( приклад LaTeX коду ) використовуй одинарний знак $ приклад LaTeX коду $

SIMILAR_EXAMPLES:
{examples}
"""
example_formatting = """
Тематика:
{subject}

Проблема:
{problem}

Вирішення:
{solution}

Відповідь:
{answer}

"""

In [30]:
import re

def evaluate_correctness(generated_answer, expected_answer, llm):
    prompt = f"""
    Оцініть правильність відповіді:
    - Згенерована відповідь: {generated_answer}
    - Очікувана відповідь: {expected_answer}

    Майте на увазі, що згенерована відповідь включає розвязок та хід задачі, вам потрібно взяти до уваги лише кінцеву відповідь та порівняти її з очікуваною.
    Чи відповідає згенерована відповідь очікуваній? Напишіть "10", якщо так, або "0", якщо ні.
    """
    response = llm(prompt)
    return int(response.strip())

def evaluate_solution_similarity(generated_solution, expected_solution, llm):
    prompt = f"""
    Оцініть схожість між рішеннями:
    - Згенероване рішення: {generated_solution}
    - Очікуване рішення: {expected_solution}

    Оцініть схожість на шкалі від 0 до 10, де 0 — зовсім не схоже, а 10 — ідентичне.
    """
    response = llm(prompt)
    try:
        return float(response.strip())
    except ValueError:
        return 0.0  # Return 0 if parsing fails


def llm(prompt):
    model = "gpt-4o-mini"
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "developer", "content": "You are useful assistant."},
            {
                "role": "user",
                "content": prompt
            }
        ]
    )
    response = completion.choices[0].message.content

    return response

def forward_pass(user_input):
    model = "gpt-4o-mini"

    similar_examples_df = similarity_search(
        query=user_input,
        top_k=15,
        similarity_threshold=0.75
    )

    similar_examples_df.loc[:, "example_text"] = similar_examples_df.apply(lambda row: example_formatting.format(
        subject=row["subject_translated"],
        problem=row["problem_translated"],
        solution=row["solution_translated"],
        answer=row["answer"]
    ), axis=1)
    examples_text = ""

    for example in similar_examples_df.loc[:, "example_text"]:
        examples_text += re.sub(r'\[asy\].*?\[/asy\]', '', example, flags=re.DOTALL)

    system_prompt_formatted = system_prompt.format(
        examples=examples_text
    )

    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "developer", "content": system_prompt_formatted},
            {
                "role": "user",
                "content": user_input
            }
        ]
    )
    response = completion.choices[0].message.content

    return response


In [31]:
import mlflow

mlflow.set_registry_uri("http://127.0.0.1:5000")

for i in range(len(test_df)):
    problem_translated = test_df.loc[i, "problem_translated"]
    solution_translated = test_df.loc[i, "solution_translated"]
    expected_answer = test_df.loc[i, "answer"]
    generated_answer = forward_pass(problem_translated)

    correctness_score = evaluate_correctness(generated_answer, expected_answer, llm)
    similarity_score = evaluate_solution_similarity(generated_answer, solution_translated, llm)

    # Log metrics in MLflow
    with mlflow.start_run(run_name=problem_translated[:35] + "..."):
        mlflow.log_metric("correctness", correctness_score)
        mlflow.log_metric("solution_similarity", similarity_score)
        mlflow.log_text(problem_translated, "problem_translated.txt")
        mlflow.log_text(solution_translated, "solution_translated.txt")
        mlflow.log_text(expected_answer, "expected_answer.txt")
        mlflow.log_text(generated_answer, "generated_answer.txt")


/var/folders/gg/1kd93k4x47q5_gj92ljtjf000000gq/T/ipykernel_39438/3683313276.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  similar_examples_df.loc[:, "example_text"] = similar_examples_df.apply(lambda row: example_formatting.format(
/var/folders/gg/1kd93k4x47q5_gj92ljtjf000000gq/T/ipykernel_39438/3683313276.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  similar_examples_df.loc[:, "example_text"] = similar_examples_df.apply(lambda row: example_formatting.format(
/var/folders/gg/1kd93k4x47q5_gj92l

ValueError: invalid literal for int() with base 10: 'Згенерована відповідь містить систему розрахунків, яка призводить до отримання площі $S$ як $\\frac{1}{2}$. \n\nОднак, очікувана відповідь - це $\\frac{1}{4}$. Це вказує на те, що згенерована відпові